In [7]:
import dill 
import torch 
import torch.nn as nn
import numpy as np
from four_room.shortest_path import find_all_action_values
from four_room.env import FourRoomsEnv
from four_room.wrappers import gym_wrapper
import numpy as np
import dill
import gymnasium as gym
from four_room.utils import obs_to_state
from four_room.arch import CNN
from tqdm import tqdm

### Setting up the datasets

In [2]:
gym.register('MiniGrid-FourRooms-v1', FourRoomsEnv)
size = 19
with open('configs/train.pl', 'rb') as file:
    train_config = dill.load(file)

In [3]:
env = gym_wrapper(gym.make(
        'MiniGrid-FourRooms-v1', 
        agent_pos= train_config['agent positions'],
        goal_pos = train_config['goal positions'],
        doors_pos = train_config['topologies'],
        agent_dir = train_config['agent directions'],
        size=size
    ),
    original_obs=True
)

c:\Users\garga\Documents\Uni\four_room\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.height to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.height` for environment variables or `env.get_wrapper_attr('height')` that will search the reminding wrappers.
  logger.warn(
c:\Users\garga\Documents\Uni\four_room\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.width to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.width` for environment variables or `env.get_wrapper_attr('width')` that will search the reminding wrappers.
  logger.warn(


In [4]:
class Dataset:
    
    def __init__(self):
        self.X = []
        self.Y = []
        self.states = []
        
    def add(self, x, y, state):
        self.X.append(x)
        self.Y.append(y)
        self.states.append(state)
        
        assert len(self.X) == len(self.Y)
    
    def __getitem__(self, index):
        return (self.X[index], self.Y[index])
    
    def __len__(self):
        return len(self.X)
    

In [5]:
argmax_dataset = Dataset()

for i in tqdm(range(len(train_config['topologies']))):
    obs, _ = env.reset()
    done = False
    while not done:
        state = obs_to_state(obs)
        q = find_all_action_values(state[:2], state[2], state[3:5], state[5:], 0.99, size)
        action = np.array(q).argmax()
        argmax_dataset.add(obs, q, state)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
with open('action_values/argmax_dataset.pl', 'wb') as file:
    dill.dump(argmax_dataset, file)

  0%|          | 0/200 [00:00<?, ?it/s]c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):
100%|██████████| 200/200 [00:54<00:00,  3.70it/s]


In [6]:
random_dataset = Dataset()

for i in tqdm(range(len(train_config['topologies']))):
    obs, _ = env.reset()
    done = False
    while not done:
        state = obs_to_state(obs)
        q = find_all_action_values(state[:2], state[2], state[3:5], state[5:], 0.99, size)
        action = env.action_space.sample()
        random_dataset.add(obs, q, state)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
with open('action_values/random_dataset.pl', 'wb') as file:
    dill.dump(random_dataset, file)

  0%|          | 0/200 [00:00<?, ?it/s]c:\Users\garga\Documents\Uni\four_room\four_room\shortest_path.py:152: RuntimeWarning: overflow encountered in scalar subtract
  for p in nx.all_shortest_paths(graph, source=(*agent_pos, (agent_dir - 1)%4), target=(*goal_pos, dir)):
100%|██████████| 200/200 [05:38<00:00,  1.69s/it]


### Regression part

In [ ]:
class RegressionModel(nn.Module):
    
    def __init__(
        self, 
        obsservation_space, 
        *args, 
        **kwargs
    ):
        super(RegressionModel, self).__init__(*args, **kwargs)
        
        
        